# Economy in East Africa
***
<img src="https://images.unsplash.com/photo-1633158829875-e5316a358c6f?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=3570&q=80" alt="Money Jar" style="width:500px;">

<a id="WebScraping"><h2>Getting Data</h2></a>
***
<a id="ScraperImport"><h3>Importing Libraries</h3></a>